# San Francisco city employee salary data

<br>
<img src="img/san_francisco.png">
<br>

Um dos conjuntos de dados mais populares que as cidades normalmente lançam é sua estrutura salarial.

Foi processado e enviado o conjunto de dados de salários do SF para entender melhor como São Francisco paga a seus funcionários e o que isso pode nos informar sobre como a sede da cidade está sendo adminstrada.

Inicilamente, pede-se a média salarial das 10 profissões com maior incidência no relatório, considerando o ano de 2014, com uma visualização de sua distribuição.

Alguns ângulos potencialmente interessantes de exploração:

- Das 10 profissões com maior incidência, quantos trabalhadores ganham acima e abaixo da média?
- Como o orçamento é alocado com base em diferentes grupos e responsabilidades?

In [ ]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [ ]:
salary = pd.read_csv(r'../../99 Datasets/SanFrancisco_salaries.csv.zip')
salary.info()

In [ ]:
salary.Year.value_counts()

In [ ]:
salary = salary[salary['Year'] == 2014]
salary.info()

In [ ]:
salary.JobTitle.value_counts(3).nlargest(10)

In [ ]:
columns = salary.JobTitle.value_counts(3).nlargest(10).index
columns

In [ ]:
keys = []
values = []
salaries_list = []

for column in columns:
    keys.append(column)
    values.append(round(salary[salary['JobTitle'] == column]['TotalPay'].mean(),2))
    
salaries_list = list(zip(keys, values))
salaries_list

In [ ]:
df_salaries = pd.DataFrame(salaries_list,columns=['profession','salary'])
df_salaries.sort_values(by=['salary'],ascending=False)

In [ ]:
from matplotlib import pyplot
%matplotlib inline

bins = 200

from matplotlib.pyplot import figure
figure(figsize=(15, 10), dpi=80)

for c in columns:
    ax = pyplot.hist(salary[salary['JobTitle'] == c]['TotalPay'], bins, alpha=0.5, label=c, range=(0,300000))

pyplot.legend(loc='upper right')
pyplot.show()

## Quantos trabalhadores ganham acima e abaixo da média na função de Registered Nurse? E na de Deputy Sheriff?


In [ ]:
salary.info()

In [ ]:
salary[salary.JobTitle == 'Deputy Sheriff']['TotalPay'].hist(bins=50)

In [ ]:
x = salary[salary.JobTitle == 'Deputy Sheriff']['TotalPay']

mu = np.mean(x)
std = np.std(x)
mu,std

In [ ]:
import scipy
from scipy import stats

In [ ]:
scipy.stats.norm(mu, std).pdf(2*std)

In [ ]:
scipy.stats.norm.interval(alpha=0.95,loc=mu,scale=std)

Como a distribuição já tem um perfil normal, vamos normalizar os dados e fazer a inferência

In [ ]:
sheriff = salary[salary.JobTitle == 'Deputy Sheriff']
sheriff.head()

In [ ]:
sheriff['norm_pay'] = (sheriff['TotalPay'] - sheriff['TotalPay'].mean()) / sheriff['TotalPay'].std()
sheriff['norm_pay'].hist(bins=50)

In [ ]:
len(sheriff['norm_pay'])

In [ ]:
x = sheriff['norm_pay']

mu = np.mean(x)
std = np.std(x)
mu,std

In [ ]:
# Verificando o Z-score

scipy.stats.norm.interval(alpha=0.95,loc=mu,scale=std)

In [ ]:
# Verificando o p-values

scipy.stats.norm(mu, std).pdf(2*std)

### Deputy Sheriff que ganaham abaixo da média salarial

In [ ]:
sheriff[sheriff['norm_pay']<scipy.stats.norm.interval(alpha=0.95,loc=mu,scale=std)[0]]

In [ ]:
len(sheriff[sheriff['norm_pay']<scipy.stats.norm.interval(alpha=0.95,loc=mu,scale=std)[0]])

Temos 23 profissionais que ganham abaixo da média salarial

### Deputy Sheriff que ganham acoma da média salarial

In [ ]:
sheriff[sheriff['norm_pay']>scipy.stats.norm.interval(alpha=0.95,loc=mu,scale=std)[1]]

In [ ]:
len(sheriff[sheriff['norm_pay']>scipy.stats.norm.interval(alpha=0.95,loc=mu,scale=std)[1]])

Temos 12 profissionais que ganham acima da média salarial

## Como o orçamento é alocado com base em diferentes grupos e responsabilidades?

In [ ]:
salary.JobTitle.unique()